# Fine Tuning Large Language Model - Model

In this workshop, you will learn how to fine tune the prompts and the LLMs to enhance and improves its response.

In [1]:
# Import libraries
import torch, time
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, GenerationConfig, TrainingArguments

from peft import PeftModel, LoraConfig, get_peft_model, TaskType

In [4]:
# TODO: Load and explore the following datasets

dataset_name = "knkarthick/dialogsum"
model_name = "google/flan-t5-small"
model_name = "google/flan-t5-base"

dataset = load_dataset(dataset_name)


In [6]:
# TODO: Print a record
print(dataset.shape)
print(dataset)


{'train': (12460, 4), 'validation': (500, 4), 'test': (1500, 4)}
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


## Fine tuning the LLM model

In this workshop we will be turning the <code>google/flan-t5-base</code> model.

In [7]:
# Utility function to dump a model's tunable parameters

def print_trainable_model_params(model):
   trainable_model_params = 0
   all_model_params = 0
   for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
         trainable_model_params += param.numel()
   return f"Trainable parameters: {trainable_model_params}\nTotal parameters: {all_model_params}\nPercentable of trainable parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [8]:
# TODO: Load model
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
# TODO: Print number of trainable parameters
print_trainable_model_params(original_model)

'Trainable parameters: 247577856\nTotal parameters: 247577856\nPercentable of trainable parameters: 100.00%'

In [11]:
idx = 100
print(dataset['train'][idx]['dialogue'])
print('----------')
print(dataset['train'][idx]['summary'])

#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.
----------
#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.


### Preprocess the dialogue dataset

We will train the model to summarize dialogue by creating a dialogue-summary pair for the LLM to process. The dialogue is the training data and the summary is the label. This is supervized learning.

The prompt will be as follows

```
Summarize the following dialogue.\n
\n
Fred: ...\n
Barney: ...\n
\n
Summary:\n
Summary of the conversation between Fred and Barney
```

The prompt and the summary will be tokenized for the LLM

In [13]:
# Utitlity function to prepare the data for training 
# Tokenize function
def tokenize_fn(data):
   start_prompt = 'Summarize the following dialogue.\n\n'
   end_prompt = '\n\nSummary:'
   prompt = [ start_prompt + d + end_prompt for d in data['dialogue'] ]
   summary = data['summary']
   data['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
   data['labels'] = tokenizer(summary, padding="max_length", truncation=True, return_tensors="pt").input_ids
   return data


In [14]:
# TODO: prepare the data for training with the tokenize_fn function
tokenized_dataset = dataset.map(
   tokenize_fn, batched=True
)



In [19]:
for k, v in enumerate(tokenized_dataset['train'][idx]):
   #print(f'key = {k}')
   print(f'value = {v}')
   print(f'{tokenized_dataset["train"][idx][v]}')
   print('---------------')

value = id
train_100
---------------
value = dialogue
#Person1#: I have a problem with my cable.
#Person2#: What about it?
#Person1#: My cable has been out for the past week or so.
#Person2#: The cable is down right now. I am very sorry.
#Person1#: When will it be working again?
#Person2#: It should be back on in the next couple of days.
#Person1#: Do I still have to pay for the cable?
#Person2#: We're going to give you a credit while the cable is down.
#Person1#: So, I don't have to pay for it?
#Person2#: No, not until your cable comes back on.
#Person1#: Okay, thanks for everything.
#Person2#: You're welcome, and I apologize for the inconvenience.
---------------
value = summary
#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.
---------------
value = topic
cable
---------------
value = input_ids
[12198, 1635, 1737, 8, 826, 7478, 5, 1713, 345, 13515, 536, 4663, 10, 27, 43, 3, 9, 682, 28, 82, 4807, 5, 171

In [20]:
text = tokenizer.decode(tokenized_dataset['train'][idx]['labels'])
print(text)

#Person1# has a problem with the cable. #Person2# promises it should work again and #Person1# doesn't have to pay while it's down.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
# TODO: Verify prepared data



In [21]:
# TODO: Remove id, dialogue, summary and topic columns from dataset. We only want input_ids and labels
tokenized_dataset = tokenized_dataset.remove_columns(
   ['id', 'dialogue', 'summary', 'topic']
)

In [22]:
# TODO: Verify dataset again
for k, v in enumerate(tokenized_dataset['train'][idx]):
   #print(f'key = {k}')
   print(f'value = {v}')
   print(f'{tokenized_dataset["train"][idx][v]}')
   print('---------------')

value = input_ids
[12198, 1635, 1737, 8, 826, 7478, 5, 1713, 345, 13515, 536, 4663, 10, 27, 43, 3, 9, 682, 28, 82, 4807, 5, 1713, 345, 13515, 357, 4663, 10, 363, 81, 34, 58, 1713, 345, 13515, 536, 4663, 10, 499, 4807, 65, 118, 91, 21, 8, 657, 471, 42, 78, 5, 1713, 345, 13515, 357, 4663, 10, 37, 4807, 19, 323, 269, 230, 5, 27, 183, 182, 8032, 5, 1713, 345, 13515, 536, 4663, 10, 366, 56, 34, 36, 464, 541, 58, 1713, 345, 13515, 357, 4663, 10, 94, 225, 36, 223, 30, 16, 8, 416, 1158, 13, 477, 5, 1713, 345, 13515, 536, 4663, 10, 531, 27, 341, 43, 12, 726, 21, 8, 4807, 58, 1713, 345, 13515, 357, 4663, 10, 101, 31, 60, 352, 12, 428, 25, 3, 9, 998, 298, 8, 4807, 19, 323, 5, 1713, 345, 13515, 536, 4663, 10, 264, 6, 27, 278, 31, 17, 43, 12, 726, 21, 34, 58, 1713, 345, 13515, 357, 4663, 10, 465, 6, 59, 552, 39, 4807, 639, 223, 30, 5, 1713, 345, 13515, 536, 4663, 10, 16036, 6, 2049, 21, 762, 5, 1713, 345, 13515, 357, 4663, 10, 148, 31, 60, 2222, 6, 11, 27, 22103, 21, 8, 25741, 5, 20698, 10, 1, 0, 0

### Tune model with pre-processed dataset

We will use [<code>Trainer</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#api-reference%20][%20transformers.Trainer) to train the original model. The training result will be written out. The trainer will be configure with [<code>TrainingArgument</code>](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [23]:
# CUDA information

print('CUDA available: ', torch.cuda.is_available())
if torch.cuda.is_available():
   print('B16 supported: ', torch.cuda.is_bf16_supported())
   torch.cuda.set_device(0)
   print('Current device: ', torch.cuda.current_device())
   print('CUDA device name: ', torch.cuda.get_device_name(0))

CUDA available:  False


## Fine tuning the LLM Model with Low-Rank Adaptation (LoRA) / Parameter Efficient Fine Tuning (PEFT)

We will add a LoRA adapter to the LLM (flan-t5-base) and train the adapter. The original LLM will be frozen. The adapter can be combined with the original LLM during inferencing. 

In [24]:
print(original_model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [25]:
# TODO: Configure LoRA
lora_config = LoraConfig(
   r=16,
   lora_alpha=16,
   target_modules=['q', 'v'],
   task_type=TaskType.SEQ_2_SEQ_LM   
)

In [26]:
# TODO: Add LoRA to the LLM model to be trained
lora_model = get_peft_model(original_model, lora_config)

In [27]:
# TODO: Print number of parameters, compare LoRA to the original model
print(print_trainable_model_params(lora_model))

Trainable parameters: 1769472
Total parameters: 249347328
Percentable of trainable parameters: 0.71%


In [ ]:
print(lora_model)

In [30]:
# TODO: Train model with LoRA
output_dir = f'peft-dialog-{str(int(time.time()))}'
lora_training_args = TrainingArguments(
   output_dir=output_dir,
   auto_find_batch_size=True,
   learning_rate=1e-3, # 0.0001
   num_train_epochs=1
)

In [31]:
# TODO: Create trainer and train model
lora_trainer = Trainer(
   model = lora_model,
   args = lora_training_args,
   train_dataset = tokenized_dataset['train'],
   eval_dataset = tokenized_dataset['validation']
)

In [ ]:
# TODO: train - cannot run
lora_trainer.train()

# Save the model and tokenizer
lora_trainer.save_pretrained("mymodel")
tokenizer.save_pretrained("mymodel")

# Upload to Hugging Face


### Use a trained LoRA model

The training will take a few hours and over many iterations.

For the purpose of this workshop we use a save model [intotheverse/peft-dialogue-summary-checkpoint](https://huggingface.co/intotheverse/peft-dialogue-summary-checkpoint).

In [32]:
#TODO: Load the original model and add the pre-trained LoRA adaptation to the model
peft_dialogue_summary_checkpoint = 'intotheverse/peft-dialogue-summary-checkpoint'

# Load base model and tokenizer
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenzer = AutoTokenizer.from_pretrained(model_name)

# Load the LoRA over base_model
lora_model = PeftModel.from_pretrained(base_model, peft_dialogue_summary_checkpoint,
      torch_dtype=torch.bfloat16)

# Load the original model
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

## Evaluate LoRA model

In [ ]:
# TODO: Evaluate LoRA model against the original 



In [33]:
# Prepare data for evaluation
dialogues = []
summaries = []
orig_model_summaries = []
lora_model_summaries = []
config = GenerationConfig(max_new_tokens=200)

for i in range(5):
   print(f'i = {i}')
   d = dataset['test'][i]['dialogue']
   s = dataset['test'][i]['summary']
   prompt = f"Summarize the following conversation.\n\n{d}\n\nSummary:"
   tokenized_prompt = tokenizer(prompt, return_tensors='pt').input_ids
   orig_resp = original_model.generate(input_ids=tokenized_prompt, generation_config=config)
   orig_resp_text = tokenizer.decode(orig_resp[0], skip_special_tokens=True)
   lora_resp = lora_model.generate(input_ids=tokenized_prompt, generation_config=config)
   lora_resp_text = tokenizer.decode(lora_resp[0], skip_special_tokens=True)

   summaries.append(s)
   orig_model_summaries.append(orig_resp_text)
   lora_model_summaries.append(lora_resp_text)

zipped_summaries = list(zip(summaries, orig_model_summaries, lora_model_summaries))
df = pd.DataFrame(zipped_summaries, columns=['label', 'original_model_summary', 'lora_model_summary'])
df

i = 0
i = 1
i = 2
i = 3
i = 4


,label,original_model_summary,lora_model_summary
0,Ms. Dawson helps #Person1# to write a memo to ...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,#Person2# got stuck in traffic and #Person1# s...


### Evaluate models with ROUGE/Bleu metrics

Recall-Oriented Understudy for Gisting Evaluate ([ROUGE](https://pub.aimind.so/unveiling-the-power-of-rouge-metrics-in-nlp-b6d3f96d3363)) is a set of metrics used to evaluate the quality of machine-generated text, such as summaries and translations. ROUGE metrics compare the generated text to a human-written reference and measure the overlap between the two. 

The metrics range between 0 and 1, with higher scores indicating higher similarity between the baseline and generated text.

In [35]:
# TODO: create ROUGE metrics
rouge = evaluate.load('rouge')

In [36]:
# TODO: Evaluate the original model's result
orig_model_results = rouge.compute(
   references=summaries,
   predictions=orig_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)

lora_model_results = rouge.compute(
   references=summaries,
   predictions=lora_model_summaries,
   use_aggregator=True,
   use_stemmer=True
)

print('Original model results')
print(orig_model_results)

print('Lora model results')
print(lora_model_results)

Original model results
{'rouge1': np.float64(0.17391941391941393), 'rouge2': np.float64(0.03820816864295125), 'rougeL': np.float64(0.13364468864468865), 'rougeLsum': np.float64(0.13364468864468865)}
Lora model results
{'rouge1': np.float64(0.34193513803269904), 'rouge2': np.float64(0.1024924201890494), 'rougeL': np.float64(0.2728511771470072), 'rougeLsum': np.float64(0.2728511771470072)}


In [ ]:
# TODO: Evaluate with Bleu metrics
